<h1>New Experiment 2: Random rejection of motion-free subjects</h1>
In this notebook, random rejection tests were conducted using data from S1 in the still condition. The generalisation tests were conducted over S2. Each time we fix the number of retained volumes N. We randomly select N volumes from a subject's data, then we train our proposed model over this sample. At testing, we applied the same rejection scheme to S2, then test over the subset of S2.</br>
We would like to show this proposed model is robust, by which it is independent of 1. The proportion of discarded volumes and 2. The different combinations of b-values.</br>
<h2>Packages</h2>

In [1]:
import nibabel as nib
import numpy as np
"""
packages that does conventional model fitting
"""
import amico
"""
packages that generate train/test dataset
"""
from FormatData import generate_data, parser as data_parser
"""
packages that trains network
"""
from Training import train_network
from utils.model import parser as model_parser
"""
packages that test network
"""
from Testing import test_model
"""
packages that produce the rejection shceme
"""
from filter_qa import parser as filter_parser, load_eddy
"""
package to store the intermediate result
"""
import pickle

2022-07-14 13:06:49.180233: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
"""
packages that handle graphs
"""
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap
from skimage.metrics import structural_similarity as compare_ssim
from utils import calc_ssim
%matplotlib inline
def plot_loss(cmd):
    """
    A function that used to plot the loss curve for the trained network.
    Args:
        cmd: String, the command line in the terminal
    """
    args = model_parser().parse_args(cmd.split())
    history = train_network(args)
    # plt.plot(history.history['loss'])
    # plt.plot(history.history['val_loss'])
    # plt.title('model loss')
    # plt.ylabel('loss')
    # plt.xlabel('epoch')
    # plt.legend(['train', 'validation'], loc='upper left')
    # plt.show()

def show_slices(slices, grayscale=True):
    """
    Function to display the slices

    Args:
        slices (list): a list of 2d ndarray that contains the data to be displayed
        grayscale (bool, optional): True, if diplay grayscale img. Defaults to True.
    """    
    fig, axes = plt.subplots(1, len(slices), figsize=(10,10))
    cax = fig.add_axes([0, 0, .3, .3])
    for i, slice in enumerate(slices):
        # use grayscale for displaying ref and pred imgs:
        if grayscale:
            cmap = plt.get_cmap('gray')
            cmaplist = [cmap(i) for i in range(cmap.N)]
            cmap = LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            bounds = np.arange(0, 1.0, .01)
            idx = np.searchsorted(bounds, 0)
            bounds = np.insert(bounds, idx, 0)
            norm = BoundaryNorm(bounds, cmap.N)
            im = axes[i].imshow(slice.T, cmap=cmap, origin="lower", interpolation='none', norm=norm)
        else:
            # define the colormap
            cmap = plt.get_cmap('bwr')
            # extract all colors from the .jet map
            cmaplist = [cmap(i) for i in range(cmap.N)]
            # create the new map
            cmap = LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            # define the bins and normalize and forcing 0 to be part of the colorbar
            # define the min and max to be -1 and +1 respectively
            bounds = np.arange(-0.5, 0.5, .01)
            idx = np.searchsorted(bounds, 0)
            bounds = np.insert(bounds, idx, 0)
            norm = BoundaryNorm(bounds, cmap.N)
            im = axes[i].imshow(slice.T, cmap=cmap, origin="lower", interpolation='none', norm=norm)
        fig.colorbar(im, cax=cax, orientation='vertical')

def scale(img):
    # for i in range(img.shape[0]):
    #     for j in range(img.shape[1]):
    #         img[i][j] = ((img[i][j]+1)/2)*255
    return img

def compare_simi(pred, ref):
    return calc_ssim(pred, ref)

In [3]:
def visualise(ref_ndi, ref_odi, ref_fwf, retained_vol, subject, model, layer, affine1, affine2, affine3):
    """
    Function to visualise the imgs and difference maps

    Args:
        ref_ndi (ndarray): the reference NDI data
        ref_odi (ndarray): the reference ODI data
        ref_fwf (ndarray): the reference FWF data
        retained_vol (int): the number of volumes retained after rejection
        subject (string): the subject that is examined
        model (string): the model used
        layer (int): the number of layers for the network
    """
    patch = 3
    if model == 'fc1d':
        patch = 1

    print(patch)
    print(model)
    # visualise the ref imgs
    refNDI0 = ref_ndi[26, :, :]
    refNDI1 = ref_ndi[:, 30, :]
    refNDI2 = ref_ndi[:, :, 16]
    show_slices([refNDI0, refNDI1, refNDI2])
    plt.suptitle("Center slices for NDI reference image")

    refODI0 = ref_odi[26, :, :]
    refODI1 = ref_odi[:, 30, :]
    refODI2 = ref_odi[:, :, 16]
    show_slices([refODI0, refODI1, refODI2])
    plt.suptitle("Center slices for ODI reference image")

    refFWF0 = ref_fwf[26, :, :]
    refFWF1 = ref_fwf[:, 30, :]
    refFWF2 = ref_fwf[:, :, 16]
    show_slices([refFWF0, refFWF1, refFWF2])
    plt.suptitle("Center slices for FWF reference image")

    # visualise the pred imgs produced at varied input size
    ndi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDI.nii'
    odi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODI.nii'
    fwf_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWF.nii'
    ndi_img = nib.load(ndi_path)
    ndi_data = ndi_img.get_fdata()
    odi_img = nib.load(odi_path)
    odi_data = odi_img.get_fdata()
    fwf_img = nib.load(fwf_path)
    fwf_data = fwf_img.get_fdata()

    ndi0 = ndi_data[26, :, :]
    ndi1 = ndi_data[:, 30, :]
    ndi2 = ndi_data[:, :, 16]
    show_slices([ndi0, ndi1, ndi2])
    plt.suptitle('Center slices for NDI predicted image by '+model+', input size='+str(retained_vol))
    (score, ndidiff) = compare_ssim(ndi_data, ref_ndi, full=True)
    print(str(retained_vol)+'input size the ssim score for ndi is: ' + str(score))

    odi0 = odi_data[26, :, :]
    odi1 = odi_data[:, 30, :]
    odi2 = odi_data[:, :, 16]
    show_slices([odi0, odi1, odi2])
    plt.suptitle('Center slices for ODI predicted image by '+model+', input size='+str(retained_vol))
    (score, odidiff) = compare_ssim(odi_data, ref_odi, full=True)
    print(str(retained_vol)+'input size the ssim score for odi is: ' + str(score))

    fwf0 = fwf_data[26, :, :]
    fwf1 = fwf_data[:, 30, :]
    fwf2 = fwf_data[:, :, 16]
    show_slices([fwf0, fwf1, fwf2])
    plt.suptitle('Center slices for FWF predicted image by '+model+', input size='+str(retained_vol))
    (score, fwfdiff) = compare_ssim(fwf_data, ref_fwf, full=True)
    print(str(retained_vol)+'input size the ssim score for fwf is: ' + str(score))

    # plot the difference map between the imgs by the lib
    ndidiff0 = scale(refNDI0 - ndi0)
    ndidiff1 = scale(refNDI1 - ndi1)
    ndidiff2 = scale(refNDI2 - ndi2)
    show_slices([ndidiff0, ndidiff1, ndidiff2], grayscale=False)
    plt.suptitle("Difference map NDI")

    diff_img_np = ref_ndi - ndi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine1)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDI_difference.nii')

    odidiff0 = scale(refODI0 - odi0)
    odidiff1 = scale(refODI1 - odi1)
    odidiff2 = scale(refODI2 - odi2)
    show_slices([odidiff0, odidiff1, odidiff2], grayscale=False)
    plt.suptitle("Difference map ODI")

    diff_img_np = ref_odi - odi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine2)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODI_difference.nii')

    fwfdiff0 = scale(refFWF0 - fwf0)
    fwfdiff1 = scale(refFWF1 - fwf1)
    fwfdiff2 = scale(refFWF2 - fwf2)
    show_slices([fwfdiff0, fwfdiff1, fwfdiff2], grayscale=False)
    plt.suptitle("Difference map FWF")

    diff_img_np = ref_fwf - fwf_data
    diff_img = nib.Nifti1Image(diff_img_np, affine3)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWF_difference.nii')

<h2>The class for parsing command line arguments</h2>

In [4]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

------------------------
<h2>Random Rejection Scheme Setup</h2>
Let N to be the tested number of retained volumes. For N, 100 subsampled schemes were drawn randomly from the full scheme (with the first b=0 volume and at least two different b values always included). Each subsampled schem was used to evaluate both techniques. For both 3D CNN and AMICO, N are 60, 40 and 30. We would give further undersampled scheme to 3D CNN, N are 20, 16 and 12.

In [5]:
bvals = '0 3000 1000 2000 2000 1000 2000 3000 1000 2000 3000 1000 2000 3000 1000 2000 3000 0 1000 3000 2000 1000 2000 3000 1000 2000 3000 1000 2000 3000 1000 2000 3000 0 1000 3000 2000 1000 1000 2000 3000 2000 3000 1000 2000 3000 1000 2000 3000 0 1000 2000 3000 1000 2000 3000 1000 2000 3000 1000 2000 3000 2000 1000 3000 0 1000 3000 2000 3000 1000 2000 1000 2000 3000 3000 1000 2000 1000 2000 3000 0 1000 3000 2000 1000 2000 2000 1000 3000 3000 1000 2000 3000 3000 1000'
bvals = np.asarray(bvals.split())
b1000 = np.where(bvals=='1000')
b2000 = np.where(bvals=='2000')
b3000 = np.where(bvals=='3000')
b0 = np.where(bvals=='0')

In [6]:
# a list contains the N for each tested retained volume
N = [60, 40, 30, 20, 16, 12]
random_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}
random_dict

{60: [], 40: [], 30: [], 20: [], 16: [], 12: []}

In [7]:
# Generated the random scheme for each tested volume
from select import select


for n in N:
    # for each retained volume, we create 100 subsamples
    for i in range(100):
        ones = np.ones(n-1) # 1 = retained
        zeros = np.zeros(96-n)

        scheme = np.random.choice(np.concatenate([ones,zeros]), 95, replace=False)
        # always select the first b=0 volume
        scheme = np.insert(scheme, 0, 1.0)
        select_index = np.where(scheme==1)
        # check if 1000 is included
        has1000 = np.intersect1d(select_index, b1000)
        has2000 = np.intersect1d(select_index, b2000)
        has3000 = np.intersect1d(select_index, b3000)
        if len(has1000) == 0:
            scheme[b1000[0]] = 1
            if len(has2000) > 1:
                # remove a 2000 b val volume
                scheme[has2000[0]] = 0
            else:
                scheme[has3000[0]] = 0
        if len(has2000) == 0:
            scheme[b2000[0]] = 1
            if len(has1000) > 1:
                # remove a 2000 b val volume
                scheme[has1000[0]] = 0
            else:
                scheme[has3000[0]] = 0
        if len(has3000) == 0:
            scheme[b3000[0]] = 1
            if len(has1000) > 1:
                # remove a 2000 b val volume
                scheme[has1000[0]] = 0
            else:
                scheme[has2000[0]] = 0
        scheme = ' '.join(map(str, scheme))
        random_dict[n].append(scheme)

In [8]:
# save the rejection scheme
random_60_file = 'random_60.pickle'
random_40_file = 'random_40.pickle'
random_30_file = 'random_30.pickle'
random_20_file = 'random_20.pickle'
random_16_file = 'random_16.pickle'
random_12_file = 'random_12.pickle'

with open(random_60_file, 'wb') as handle:
    pickle.dump(random_dict[60], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(random_40_file, 'wb') as handle:
    pickle.dump(random_dict[40], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(random_30_file, 'wb') as handle:
    pickle.dump(random_dict[30], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(random_20_file, 'wb') as handle:
    pickle.dump(random_dict[20], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(random_16_file, 'wb') as handle:
    pickle.dump(random_dict[16], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(random_12_file, 'wb') as handle:
    pickle.dump(random_dict[12], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
cnn_rmse_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}
cnn_ssim_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}

----------------------------
<h2>Training</h2>
Hyperparameters

In [10]:
# hyperparameters for the networks
layers = 4 # the number of hidden layers; this is the optimal number of layer
lr = 0.0001 # the learning rate
patch_size = 3 # the size of patches for 2D and 3D CNN
batch = 256 # the batch size
epoch = 100 # the number of epoches for training
model = 'conv3d'
ltype = 'A'

# we firstly study subject s02, use s01 as a seperate training dataset
study_subject = 's03_still_reg' # the subject we want to study
sep_train_subject = 's01_still' # this is separate training candidate

<h4>N = 60</h4>

In [11]:
for i in range(len(random_dict[60])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[60][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[60].append(RMSE)
    cnn_ssim_dict[60].append(SSIM)

60
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3, 3, 3, 60)]     0         
_________________________________________________________________
conv3d (Conv3D)              (None, 1, 1, 1, 150)      243150    
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 1, 1, 1, 3)    

2022-07-14 13:06:50.556797: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-07-14 13:06:50.583061: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-14 13:06:50.583143: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.777GHz coreCount: 28 deviceMemorySize: 11.76GiB deviceMemoryBandwidth: 335.32GiB/s
2022-07-14 13:06:50.583164: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-07-14 13:06:50.585485: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-07-14 13:06:50.602527: I tensorflow/stream_executor/pl

The remained data has shape: (81634, 3, 3, 3, 60)


2022-07-14 13:06:55.482767: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 1057976640 exceeds 10% of free system memory.
2022-07-14 13:06:55.809428: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 528988320 exceeds 10% of free system memory.


(81634, 3, 3, 3, 60)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 3)
(81634, 1, 1, 1, 3)
Training start ...


...

2022-07-14 14:21:11.832180: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 14:21:11.852648: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 14:21:11.852913: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 14:21:11.855334: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_14_21_11
2022-07-14 14:21:11.855882: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_14_21_11/vw.trace.json.gz
2022-07-14 14:21:11.855957: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 14:21:11.858219: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_293"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_98 (InputLayer)        [(None, 84, 84, 50, 60)]  0         
_________________________________________________________________
conv3d_485 (Conv3D)          (None, 82, 82, 48, 150)   243150    
_________________________________________________________________
conv3d_486 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_487 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_488 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_97 (Dropout)         (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_489 (Conv3D)   

2022-07-14 14:22:44.754336: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 14:22:44.754463: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 14:22:45.291674: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 14:22:45.312024: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 14:22:45.312294: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 14:22:45.314766: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_14_22_45
2022-07-14 14:22:45.315296: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_14_22_45/vw.trace.json.gz
2022-07-14 14:22:45.315364: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 14:22:45.317595: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_299"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_100 (InputLayer)       [(None, 84, 84, 50, 60)]  0         
_________________________________________________________________
conv3d_495 (Conv3D)          (None, 82, 82, 48, 150)   243150    
_________________________________________________________________
conv3d_496 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_497 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_498 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_99 (Dropout)         (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_499 (Conv3D)   

In [12]:
with open('cnn_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
for i in range(len(random_dict[60])//2, len(random_dict[60])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[60][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[60].append(RMSE)
    cnn_ssim_dict[60].append(SSIM)

60
Model: "functional_301"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_101 (InputLayer)       [(None, 3, 3, 3, 60)]     0         
_________________________________________________________________
conv3d_500 (Conv3D)          (None, 1, 1, 1, 150)      243150    
_________________________________________________________________
conv3d_501 (Conv3D)          (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_502 (Conv3D)          (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_503 (Conv3D)          (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_100 (Dropout)        (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_504 (Conv3D)          (None, 1, 1, 1, 3)  

2022-07-14 14:24:18.306985: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 14:24:18.307102: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 14:24:18.987977: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 14:24:19.020784: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 14:24:19.021114: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 14:24:19.024067: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_14_24_19
2022-07-14 14:24:19.024675: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_14_24_19/vw.trace.json.gz
2022-07-14 14:24:19.024774: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 14:24:19.027243: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_305"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_102 (InputLayer)       [(None, 84, 84, 50, 60)]  0         
_________________________________________________________________
conv3d_505 (Conv3D)          (None, 82, 82, 48, 150)   243150    
_________________________________________________________________
conv3d_506 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_507 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_508 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_101 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_509 (Conv3D)   

2022-07-14 14:25:52.328345: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 14:25:52.328538: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


...

2022-07-14 15:39:19.008837: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 15:39:19.029137: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 15:39:19.029415: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 15:39:19.031868: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_15_39_19
2022-07-14 15:39:19.032404: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_15_39_19/vw.trace.json.gz
2022-07-14 15:39:19.032472: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 15:39:19.034715: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_593"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_198 (InputLayer)       [(None, 84, 84, 50, 60)]  0         
_________________________________________________________________
conv3d_985 (Conv3D)          (None, 82, 82, 48, 150)   243150    
_________________________________________________________________
conv3d_986 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_987 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_988 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_197 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_989 (Conv3D)   

2022-07-14 15:40:48.004849: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 15:40:48.004932: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 15:40:48.694182: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 15:40:48.714233: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 15:40:48.714513: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 15:40:48.717040: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_15_40_48
2022-07-14 15:40:48.717592: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_15_40_48/vw.trace.json.gz
2022-07-14 15:40:48.717663: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 15:40:48.719930: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_599"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_200 (InputLayer)       [(None, 84, 84, 50, 60)]  0         
_________________________________________________________________
conv3d_995 (Conv3D)          (None, 82, 82, 48, 150)   243150    
_________________________________________________________________
conv3d_996 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_997 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_998 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_199 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_999 (Conv3D)   

In [14]:
with open('cnn_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 40</h4>

In [15]:
for i in range(len(random_dict[40])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[40][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[40].append(RMSE)
    cnn_ssim_dict[40].append(SSIM)

40
Model: "functional_601"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_201 (InputLayer)       [(None, 3, 3, 3, 40)]     0         
_________________________________________________________________
conv3d_1000 (Conv3D)         (None, 1, 1, 1, 150)      162150    
_________________________________________________________________
conv3d_1001 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_1002 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_1003 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_200 (Dropout)        (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_1004 (Conv3D)         (None, 1, 1, 1, 3)  

2022-07-14 15:42:20.626465: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 15:42:20.626581: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 15:42:21.267284: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 15:42:21.287125: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 15:42:21.287403: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 15:42:21.289916: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_15_42_21
2022-07-14 15:42:21.290477: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_15_42_21/vw.trace.json.gz
2022-07-14 15:42:21.290546: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 15:42:21.292813: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_605"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_202 (InputLayer)       [(None, 84, 84, 50, 40)]  0         
_________________________________________________________________
conv3d_1005 (Conv3D)         (None, 82, 82, 48, 150)   162150    
_________________________________________________________________
conv3d_1006 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1007 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1008 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_201 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_1009 (Conv3D)  

2022-07-14 15:43:36.721081: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 15:43:36.721199: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


...

2022-07-14 16:43:17.049065: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 16:43:17.072095: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 16:43:17.072392: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 16:43:17.075051: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_16_43_17
2022-07-14 16:43:17.075592: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_16_43_17/vw.trace.json.gz
2022-07-14 16:43:17.075667: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 16:43:17.078006: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_893"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_298 (InputLayer)       [(None, 84, 84, 50, 40)]  0         
_________________________________________________________________
conv3d_1485 (Conv3D)         (None, 82, 82, 48, 150)   162150    
_________________________________________________________________
conv3d_1486 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1487 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1488 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_297 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_1489 (Conv3D)  

2022-07-14 16:44:32.817311: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 16:44:32.817426: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 16:44:33.623130: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 16:44:33.644433: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 16:44:33.644716: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 16:44:33.665648: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_16_44_33
2022-07-14 16:44:33.666776: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_16_44_33/vw.trace.json.gz
2022-07-14 16:44:33.667017: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 16:44:33.670714: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_899"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_300 (InputLayer)       [(None, 84, 84, 50, 40)]  0         
_________________________________________________________________
conv3d_1495 (Conv3D)         (None, 82, 82, 48, 150)   162150    
_________________________________________________________________
conv3d_1496 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1497 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1498 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_299 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_1499 (Conv3D)  

In [16]:
with open('cnn_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
for i in range(len(random_dict[40])//2, len(random_dict[40])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[40][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[40].append(RMSE)
    cnn_ssim_dict[40].append(SSIM)

40
Model: "functional_901"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_301 (InputLayer)       [(None, 3, 3, 3, 40)]     0         
_________________________________________________________________
conv3d_1500 (Conv3D)         (None, 1, 1, 1, 150)      162150    
_________________________________________________________________
conv3d_1501 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_1502 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_1503 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_300 (Dropout)        (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_1504 (Conv3D)         (None, 1, 1, 1, 3)  

2022-07-14 16:45:49.644946: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 16:45:49.645057: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 16:45:50.441171: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 16:45:50.462337: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 16:45:50.462617: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 16:45:50.465116: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_16_45_50
2022-07-14 16:45:50.465663: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_16_45_50/vw.trace.json.gz
2022-07-14 16:45:50.465749: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 16:45:50.467947: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_905"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_302 (InputLayer)       [(None, 84, 84, 50, 40)]  0         
_________________________________________________________________
conv3d_1505 (Conv3D)         (None, 82, 82, 48, 150)   162150    
_________________________________________________________________
conv3d_1506 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1507 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1508 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_301 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_1509 (Conv3D)  

2022-07-14 16:47:06.235327: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 16:47:06.235445: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


...

2022-07-14 17:47:06.830029: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 17:47:06.851373: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 17:47:06.851649: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 17:47:06.854256: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_17_47_06
2022-07-14 17:47:06.854794: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_17_47_06/vw.trace.json.gz
2022-07-14 17:47:06.854875: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 17:47:06.857157: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_1193"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_398 (InputLayer)       [(None, 84, 84, 50, 40)]  0         
_________________________________________________________________
conv3d_1985 (Conv3D)         (None, 82, 82, 48, 150)   162150    
_________________________________________________________________
conv3d_1986 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1987 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1988 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_397 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_1989 (Conv3D) 

2022-07-14 17:48:22.801833: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 17:48:22.801956: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 17:48:23.780777: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 17:48:23.801406: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 17:48:23.801676: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 17:48:23.804191: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_17_48_23
2022-07-14 17:48:23.804714: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_17_48_23/vw.trace.json.gz
2022-07-14 17:48:23.804785: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 17:48:23.807085: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_1199"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_400 (InputLayer)       [(None, 84, 84, 50, 40)]  0         
_________________________________________________________________
conv3d_1995 (Conv3D)         (None, 82, 82, 48, 150)   162150    
_________________________________________________________________
conv3d_1996 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1997 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_1998 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_399 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_1999 (Conv3D) 

In [18]:
with open('cnn_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 30</h4>

In [19]:
for i in range(len(random_dict[30])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[30][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[30].append(RMSE)
    cnn_ssim_dict[30].append(SSIM)

30
Model: "functional_1201"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_401 (InputLayer)       [(None, 3, 3, 3, 30)]     0         
_________________________________________________________________
conv3d_2000 (Conv3D)         (None, 1, 1, 1, 150)      121650    
_________________________________________________________________
conv3d_2001 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_2002 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_2003 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_400 (Dropout)        (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_2004 (Conv3D)         (None, 1, 1, 1, 3) 

2022-07-14 17:49:39.319702: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 17:49:39.319874: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 17:49:40.228677: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 17:49:40.248886: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 17:49:40.249155: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 17:49:40.251725: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_17_49_40
2022-07-14 17:49:40.252298: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_17_49_40/vw.trace.json.gz
2022-07-14 17:49:40.252374: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 17:49:40.254726: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_1205"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_402 (InputLayer)       [(None, 84, 84, 50, 30)]  0         
_________________________________________________________________
conv3d_2005 (Conv3D)         (None, 82, 82, 48, 150)   121650    
_________________________________________________________________
conv3d_2006 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2007 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2008 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_401 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_2009 (Conv3D) 

2022-07-14 17:50:48.853554: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 17:50:48.853672: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


...

2022-07-14 18:44:59.969328: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 18:45:00.026819: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 18:45:00.030632: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 18:45:00.034953: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_18_45_00
2022-07-14 18:45:00.035641: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_18_45_00/vw.trace.json.gz
2022-07-14 18:45:00.035826: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 18:45:00.039427: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_1493"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_498 (InputLayer)       [(None, 84, 84, 50, 30)]  0         
_________________________________________________________________
conv3d_2485 (Conv3D)         (None, 82, 82, 48, 150)   121650    
_________________________________________________________________
conv3d_2486 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2487 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2488 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_497 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_2489 (Conv3D) 

2022-07-14 18:46:09.367235: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 18:46:09.367309: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 18:46:10.859066: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 18:46:10.904677: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 18:46:10.905211: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 18:46:10.909015: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_18_46_10
2022-07-14 18:46:10.909687: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_18_46_10/vw.trace.json.gz
2022-07-14 18:46:10.909814: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 18:46:10.912799: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_1499"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_500 (InputLayer)       [(None, 84, 84, 50, 30)]  0         
_________________________________________________________________
conv3d_2495 (Conv3D)         (None, 82, 82, 48, 150)   121650    
_________________________________________________________________
conv3d_2496 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2497 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2498 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_499 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_2499 (Conv3D) 

In [20]:
with open('cnn_rmse_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
for i in range(len(random_dict[30])//2, len(random_dict[30])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[30][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[30].append(RMSE)
    cnn_ssim_dict[30].append(SSIM)

30
Model: "functional_1501"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_501 (InputLayer)       [(None, 3, 3, 3, 30)]     0         
_________________________________________________________________
conv3d_2500 (Conv3D)         (None, 1, 1, 1, 150)      121650    
_________________________________________________________________
conv3d_2501 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_2502 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_2503 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_500 (Dropout)        (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_2504 (Conv3D)         (None, 1, 1, 1, 3) 

2022-07-14 18:47:16.725055: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 18:47:16.725132: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 18:47:18.110384: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 18:47:18.149203: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 18:47:18.149686: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 18:47:18.153276: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_18_47_18
2022-07-14 18:47:18.153935: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_18_47_18/vw.trace.json.gz
2022-07-14 18:47:18.154046: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 18:47:18.156600: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_1505"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_502 (InputLayer)       [(None, 84, 84, 50, 30)]  0         
_________________________________________________________________
conv3d_2505 (Conv3D)         (None, 82, 82, 48, 150)   121650    
_________________________________________________________________
conv3d_2506 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2507 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2508 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_501 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_2509 (Conv3D) 

2022-07-14 18:48:27.797940: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 18:48:27.798181: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


...

2022-07-14 19:43:12.307133: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 19:43:12.332779: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 19:43:12.333100: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 19:43:12.335873: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_19_43_12
2022-07-14 19:43:12.336402: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_19_43_12/vw.trace.json.gz
2022-07-14 19:43:12.336482: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 19:43:12.338855: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_1793"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_598 (InputLayer)       [(None, 84, 84, 50, 30)]  0         
_________________________________________________________________
conv3d_2985 (Conv3D)         (None, 82, 82, 48, 150)   121650    
_________________________________________________________________
conv3d_2986 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2987 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2988 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_597 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_2989 (Conv3D) 

2022-07-14 19:44:21.490970: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 19:44:21.491099: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 19:44:22.803763: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 19:44:22.828014: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 19:44:22.828302: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 19:44:22.831016: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_19_44_22
2022-07-14 19:44:22.831575: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_19_44_22/vw.trace.json.gz
2022-07-14 19:44:22.831698: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 19:44:22.834104: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_1799"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_600 (InputLayer)       [(None, 84, 84, 50, 30)]  0         
_________________________________________________________________
conv3d_2995 (Conv3D)         (None, 82, 82, 48, 150)   121650    
_________________________________________________________________
conv3d_2996 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2997 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_2998 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_599 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_2999 (Conv3D) 

In [22]:
with open('cnn_rmse_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 20</h4>

In [23]:
for i in range(len(random_dict[20])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[20][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[20].append(RMSE)
    cnn_ssim_dict[20].append(SSIM)

20
Model: "functional_1801"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_601 (InputLayer)       [(None, 3, 3, 3, 20)]     0         
_________________________________________________________________
conv3d_3000 (Conv3D)         (None, 1, 1, 1, 150)      81150     
_________________________________________________________________
conv3d_3001 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_3002 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_3003 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_600 (Dropout)        (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_3004 (Conv3D)         (None, 1, 1, 1, 3) 

2022-07-14 19:45:31.319138: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 19:45:31.319250: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 19:45:32.568046: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 19:45:32.590600: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 19:45:32.590886: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 19:45:32.593581: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_19_45_32
2022-07-14 19:45:32.594149: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_19_45_32/vw.trace.json.gz
2022-07-14 19:45:32.594245: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 19:45:32.596660: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_1805"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_602 (InputLayer)       [(None, 84, 84, 50, 20)]  0         
_________________________________________________________________
conv3d_3005 (Conv3D)         (None, 82, 82, 48, 150)   81150     
_________________________________________________________________
conv3d_3006 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3007 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3008 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_601 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_3009 (Conv3D) 

2022-07-14 19:46:32.785733: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 19:46:32.785922: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


...

2022-07-14 20:33:41.255168: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 20:33:41.281131: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 20:33:41.281460: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 20:33:41.284255: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_20_33_41
2022-07-14 20:33:41.284782: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_20_33_41/vw.trace.json.gz
2022-07-14 20:33:41.284857: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 20:33:41.287261: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2093"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_698 (InputLayer)       [(None, 84, 84, 50, 20)]  0         
_________________________________________________________________
conv3d_3485 (Conv3D)         (None, 82, 82, 48, 150)   81150     
_________________________________________________________________
conv3d_3486 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3487 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3488 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_697 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_3489 (Conv3D) 

2022-07-14 20:34:41.841700: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 20:34:41.841836: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 20:34:43.280442: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 20:34:43.304209: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 20:34:43.304527: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 20:34:43.307233: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_20_34_43
2022-07-14 20:34:43.307762: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_20_34_43/vw.trace.json.gz
2022-07-14 20:34:43.307839: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 20:34:43.310672: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2099"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_700 (InputLayer)       [(None, 84, 84, 50, 20)]  0         
_________________________________________________________________
conv3d_3495 (Conv3D)         (None, 82, 82, 48, 150)   81150     
_________________________________________________________________
conv3d_3496 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3497 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3498 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_699 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_3499 (Conv3D) 

In [24]:
with open('cnn_rmse_20_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_20_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
for i in range(len(random_dict[20])//2, len(random_dict[20])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[20][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[20].append(RMSE)
    cnn_ssim_dict[20].append(SSIM)

20
Model: "functional_2101"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_701 (InputLayer)       [(None, 3, 3, 3, 20)]     0         
_________________________________________________________________
conv3d_3500 (Conv3D)         (None, 1, 1, 1, 150)      81150     
_________________________________________________________________
conv3d_3501 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_3502 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_3503 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_700 (Dropout)        (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_3504 (Conv3D)         (None, 1, 1, 1, 3) 

2022-07-14 20:35:39.809185: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 20:35:39.809280: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 20:35:41.213555: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 20:35:41.237186: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 20:35:41.237478: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 20:35:41.240095: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_20_35_41
2022-07-14 20:35:41.240622: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_20_35_41/vw.trace.json.gz
2022-07-14 20:35:41.240696: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 20:35:41.243369: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2105"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_702 (InputLayer)       [(None, 84, 84, 50, 20)]  0         
_________________________________________________________________
conv3d_3505 (Conv3D)         (None, 82, 82, 48, 150)   81150     
_________________________________________________________________
conv3d_3506 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3507 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3508 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_701 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_3509 (Conv3D) 

2022-07-14 20:36:42.036262: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 20:36:42.036357: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


...

2022-07-14 21:24:27.045750: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 21:24:27.087580: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 21:24:27.088028: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 21:24:27.091969: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_21_24_27
2022-07-14 21:24:27.092683: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_21_24_27/vw.trace.json.gz
2022-07-14 21:24:27.092867: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 21:24:27.095842: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2393"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_798 (InputLayer)       [(None, 84, 84, 50, 20)]  0         
_________________________________________________________________
conv3d_3985 (Conv3D)         (None, 82, 82, 48, 150)   81150     
_________________________________________________________________
conv3d_3986 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3987 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3988 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_797 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_3989 (Conv3D) 

2022-07-14 21:25:28.299860: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 21:25:28.300023: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 21:25:30.200329: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 21:25:30.236392: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 21:25:30.236835: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 21:25:30.240333: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_21_25_30
2022-07-14 21:25:30.240935: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_21_25_30/vw.trace.json.gz
2022-07-14 21:25:30.241046: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 21:25:30.243676: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2399"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_800 (InputLayer)       [(None, 84, 84, 50, 20)]  0         
_________________________________________________________________
conv3d_3995 (Conv3D)         (None, 82, 82, 48, 150)   81150     
_________________________________________________________________
conv3d_3996 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3997 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_3998 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_799 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_3999 (Conv3D) 

In [26]:
with open('cnn_rmse_20_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_20_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 16</h4>

In [27]:
for i in range(len(random_dict[16])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[16][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[16].append(RMSE)
    cnn_ssim_dict[16].append(SSIM)

16
Model: "functional_2401"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_801 (InputLayer)       [(None, 3, 3, 3, 16)]     0         
_________________________________________________________________
conv3d_4000 (Conv3D)         (None, 1, 1, 1, 150)      64950     
_________________________________________________________________
conv3d_4001 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_4002 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_4003 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_800 (Dropout)        (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_4004 (Conv3D)         (None, 1, 1, 1, 3) 

2022-07-14 21:26:30.842920: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 21:26:30.843040: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 21:26:32.663595: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 21:26:32.696387: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 21:26:32.696781: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 21:26:32.699697: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_21_26_32
2022-07-14 21:26:32.700272: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_21_26_32/vw.trace.json.gz
2022-07-14 21:26:32.700359: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 21:26:32.702980: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2405"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_802 (InputLayer)       [(None, 84, 84, 50, 16)]  0         
_________________________________________________________________
conv3d_4005 (Conv3D)         (None, 82, 82, 48, 150)   64950     
_________________________________________________________________
conv3d_4006 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4007 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4008 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_801 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_4009 (Conv3D) 

2022-07-14 21:27:30.028622: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 21:27:30.028781: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


...

2022-07-14 22:11:51.390899: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 22:11:51.409554: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 22:11:51.409828: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 22:11:51.412418: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_22_11_51
2022-07-14 22:11:51.412942: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_22_11_51/vw.trace.json.gz
2022-07-14 22:11:51.413010: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 22:11:51.415370: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2693"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_898 (InputLayer)       [(None, 84, 84, 50, 16)]  0         
_________________________________________________________________
conv3d_4485 (Conv3D)         (None, 82, 82, 48, 150)   64950     
_________________________________________________________________
conv3d_4486 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4487 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4488 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_897 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_4489 (Conv3D) 

2022-07-14 22:12:46.839525: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 22:12:46.839623: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 22:12:48.433649: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 22:12:48.452188: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 22:12:48.452461: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 22:12:48.455023: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_22_12_48
2022-07-14 22:12:48.455566: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_22_12_48/vw.trace.json.gz
2022-07-14 22:12:48.455634: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 22:12:48.457998: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2699"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_900 (InputLayer)       [(None, 84, 84, 50, 16)]  0         
_________________________________________________________________
conv3d_4495 (Conv3D)         (None, 82, 82, 48, 150)   64950     
_________________________________________________________________
conv3d_4496 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4497 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4498 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_899 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_4499 (Conv3D) 

In [28]:
with open('cnn_rmse_16_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_16_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
for i in range(len(random_dict[16])//2, len(random_dict[16])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[16][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[16].append(RMSE)
    cnn_ssim_dict[16].append(SSIM)

16
Model: "functional_2701"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_901 (InputLayer)       [(None, 3, 3, 3, 16)]     0         
_________________________________________________________________
conv3d_4500 (Conv3D)         (None, 1, 1, 1, 150)      64950     
_________________________________________________________________
conv3d_4501 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_4502 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_4503 (Conv3D)         (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_900 (Dropout)        (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_4504 (Conv3D)         (None, 1, 1, 1, 3) 

2022-07-14 22:13:43.385058: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 22:13:43.385183: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 22:13:44.991921: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 22:13:45.010661: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 22:13:45.010929: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 22:13:45.031972: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_22_13_45
2022-07-14 22:13:45.032950: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_22_13_45/vw.trace.json.gz
2022-07-14 22:13:45.033118: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 22:13:45.036745: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2705"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_902 (InputLayer)       [(None, 84, 84, 50, 16)]  0         
_________________________________________________________________
conv3d_4505 (Conv3D)         (None, 82, 82, 48, 150)   64950     
_________________________________________________________________
conv3d_4506 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4507 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4508 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_901 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_4509 (Conv3D) 

2022-07-14 22:14:39.855886: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 22:14:39.856026: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


...

2022-07-14 22:59:55.483447: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 22:59:55.512936: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 22:59:55.513292: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 22:59:55.516264: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_22_59_55
2022-07-14 22:59:55.516822: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_22_59_55/vw.trace.json.gz
2022-07-14 22:59:55.516911: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 22:59:55.519411: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2993"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_998 (InputLayer)       [(None, 84, 84, 50, 16)]  0         
_________________________________________________________________
conv3d_4985 (Conv3D)         (None, 82, 82, 48, 150)   64950     
_________________________________________________________________
conv3d_4986 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4987 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4988 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_997 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_4989 (Conv3D) 

2022-07-14 23:00:50.808628: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 23:00:50.808714: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-07-14 23:00:52.835184: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-07-14 23:00:52.863558: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-07-14 23:00:52.863904: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-07-14 23:00:52.866840: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_07_14_23_00_52
2022-07-14 23:00:52.867403: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_14_23_00_52/vw.trace.json.gz
2022-07-14 23:00:52.867490: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-07-14 23:00:52.869982: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_still_reg
Model: "functional_2999"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1000 (InputLayer)      [(None, 84, 84, 50, 16)]  0         
_________________________________________________________________
conv3d_4995 (Conv3D)         (None, 82, 82, 48, 150)   64950     
_________________________________________________________________
conv3d_4996 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4997 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_4998 (Conv3D)         (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_999 (Dropout)        (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_4999 (Conv3D) 

In [30]:
with open('cnn_rmse_16_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_16_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 12</h4>

In [31]:
# read the rejection scheme
random_12 = []
with (open('/home/vw/Desktop/IndividualProject/MedICSS2021_/Net/random_12.pickle', "rb")) as openfile:
    while True:
        try:
            random_12.append(pickle.load(openfile))
        except EOFError:
            break
random_12 = np.array(random_12[0])

FileNotFoundError: [Errno 2] No such file or directory: '/home/vw/Desktop/IndividualProject/MedICSS2021_/Net/random_12.pickle'

In [ ]:
# for i in range(len(random_dict[12])//2):
for i in range(len(random_12)//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    # movefile = random_dict[12][i]
    movefile = random_12[i]
    # movefile = movefile.split()
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[12].append(RMSE)
    cnn_ssim_dict[12].append(SSIM)

In [ ]:
with open('cnn_rmse_12_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_12_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# for i in range(len(random_dict[12])//2, len(random_dict[12])):
for i in range(len(random_12)//2, len(random_12)):
    # we have 100 movefiles for each retained tested volume
    # Training
    # movefile = random_dict[12][i]
    movefile = random_12[i]
    # movefile = movefile.split()
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[12].append(RMSE)
    cnn_ssim_dict[12].append(SSIM)

In [ ]:
with open('cnn_rmse_12_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_12_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)